## **Chatbot con `Llama2`**

---
### **Librerías**

In [1]:
import json
import numpy as np
import torch
from sentence_transformers import SentenceTransformer

import matplotlib.pyplot as plt

# from sklearn.decomposition import uPCA
import seaborn as sns
import faiss

from transformers import AutoTokenizer, AutoModelForCausalLM

/opt/miniconda3/envs/PharmAI/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PRUEBA HASTA QUE CHARLY NO TERMINE

In [2]:
def retrieve_relevant_fragments(query, top_k=5):
    """
    Recupera los fragmentos más relevantes para la consulta utilizando un modelo de similitud (ej. FAISS).

    Parámetros:
    - query (str): Consulta del usuario.
    - top_k (int): Número de fragmentos a recuperar.

    Retorna:
    - list: Lista de fragmentos relevantes.
    """
    # Simulación de la búsqueda, usando FAISS o similar.
    # Esto debería ser reemplazado por la implementación real que recupera los fragmentos relevantes.
    # Suponemos que "retrieved_fragments" es el resultado de una búsqueda en base de datos vectorial.

    retrieved_fragments = [
        {
            "medicamento": "Aspirina",
            "categoria": "efectos_secundarios",
            "texto": "Puede causar náuseas y dolor de estómago.",
        },
        {
            "medicamento": "Paracetamol",
            "categoria": "efectos_secundarios",
            "texto": "Puede causar problemas hepáticos en dosis altas.",
        },
    ]
    return retrieved_fragments

---
### **1. Cargar Modelo**

In [3]:
def load_llama_model():

    # Detectar el dispositivo disponible: CUDA, MPS (para Mac con Apple Silicon) o CPU
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"

    print("Usando dispositivo:", device)

    # Nombre del modelo a cargar (Llama-2-7b)
    model_name = "meta-llama/Llama-2-7b-hf"

    # Cargar el tokenizador
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

    # Cargar el modelo, especificando el tipo de datos y usando device_map="auto" para aprovechar la GPU
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if device in ["cuda", "mps"] else torch.float32,
        device_map="auto",
    )

    return model, tokenizer


# Cargar el modelo
model, tokenizer = load_llama_model()

Usando dispositivo: mps


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.09s/it]
Some parameters are on the meta device because they were offloaded to the disk.


---
### **2. Dar formato al contexto que recibimos de la Base de Datos Vectorial**

In [4]:
def format_context(retrieved_fragments, max_fragments=5, max_text_length=2000):
    """
    Formatea los fragmentos recuperados en un contexto para el modelo.

    Parámetros:
    - retrieved_fragments (list): Lista de fragmentos recuperados
    - max_fragments (int): Número máximo de fragmentos a utilizar
    - max_text_length (int): Longitud máxima del texto a mostrar por fragmento

    Retorna:
    - str: Contexto formateado para el modelo
    """
    context = ""
    for i, frag in enumerate(retrieved_fragments[:max_fragments]):
        medicamento = frag["medicamento"]
        categoria = frag["categoria"]
        texto = frag["texto"]

        # Limitamos la longitud del texto para evitar sobrecargar el modelo
        truncated_text = (
            texto[:max_text_length] + "..." if len(texto) > max_text_length else texto
        )

        context += f"\nFragmento {i+1}:\n"
        context += f"Medicamento: {medicamento}\n"
        context += f"Categoría: {categoria}\n"
        context += f"Información: {truncated_text}\n"

    return context

---
### **3. Construir Prompt**

In [5]:
def build_prompt(context, query):
    """
    Construye el prompt para el modelo con base en el contexto y la consulta,
    incluyendo un ejemplo de cómo debe formatear la respuesta.

    Parámetros:
    - context (str): Contexto a proporcionar al modelo
    - query (str): Consulta del usuario

    Retorna:
    - str: Prompt completo para el modelo
    """
    prompt = f"""Eres un asistente médico especializado en información sobre medicamentos.
    Basándote únicamente en la siguiente información sobre medicamentos:

    {context}

    Responde de manera clara y precisa a esta pregunta: {query}

    Ejemplo de consulta:
    Pregunta: ¿Cuáles son los efectos secundarios de la aspirina?
    Contexto: 
    Fragmento 1:
    Medicamento: Aspirina
    Categoría: reacciones_adversas
    Información: Los efectos secundarios comunes incluyen náuseas, dolor de estómago y sangrados.

    Respuesta:
    La aspirina puede causar efectos secundarios como náuseas, dolor de estómago y sangrados, según la información proporcionada en el fragmento. Si necesitas más detalles, por favor consulta la ficha técnica completa.

    Si la información proporcionada no es suficiente para responder completamente, indica qué datos faltan.

    Tu respuesta debe ser:
    1. Precisa y basada solo en el contexto proporcionado.
    2. Estructurada y fácil de entender.
    3. Sin añadir información que no esté en los fragmentos.
    4. Con referencias claras al medicamento mencionado.

    Respuesta:"""
    return prompt

---
### **4. Generar respuestas basadas en el contexto y el prompt**

In [6]:
def generate_answer(query, retrieved_fragments, tokenizer, model, max_fragments=5):
    """
    Genera una respuesta basada en los fragmentos recuperados usando LLaMA.

    Parámetros:
    - query (str): La consulta del usuario
    - retrieved_fragments (list): Lista de fragmentos recuperados
    - tokenizer: Tokenizador del modelo
    - model: Modelo generativo
    - max_fragments (int): Número máximo de fragmentos a utilizar

    Retorna:
    - str: Respuesta generada
    """
    # Formateamos el contexto usando los fragmentos recuperados
    context = format_context(retrieved_fragments, max_fragments)

    # Construimos el prompt para el modelo
    prompt = build_prompt(context, query)

    # Tokenizamos el prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Detectar el dispositivo disponible: CUDA, MPS (para Mac con Apple Silicon) o CPU
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"

    print("Usando dispositivo:", device)

    input_ids = input_ids.to(device)

    # Generamos la respuesta usando el modelo
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=len(input_ids[0]) + 500,  # Limita la longitud de salida
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
        )

    # Decodificamos la respuesta generada
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extraemos solo la parte de la respuesta después del prompt
    response = response[
        len(prompt) :
    ].strip()  # Ajuste para capturar la respuesta correctamente

    return response

---
### **5. Realizar la consulta**

In [7]:
def answer_query(query):
    """
    Realiza una consulta y genera una respuesta utilizando el modelo.

    Parámetros:
    - query (str): La consulta del usuario

    Retorna:
    - str: Respuesta generada
    """
    # Recuperamos los fragmentos relevantes para la consulta
    retrieved_fragments = retrieve_relevant_fragments(query)

    # Generamos la respuesta utilizando la función que ya tienes
    response = generate_answer(query, retrieved_fragments, tokenizer, model)

    return response

---
### **6. Ejemplo de Consulta**

In [9]:
query = "¿Cuáles son los efectos secundarios del paracetamol?"
response = answer_query(query)
print("Respuesta generada:", response)

Usando dispositivo: mps
Respuesta generada: El paracetamol puede causar problemas hepáticos en dosis altas. Por lo tanto, su uso no recomendable durante el embarazo. Además, hay que evitar tomarlo junto con alcohol o antidepresivos, ya que se incrementará el riesgo de intoxicación.
